# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time 
import urllib
from scipy.stats import linregress

from api_keys import api_key

from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

#Range of lat and long
lat_range = (-90,90)
lng_range = (-180,180)

## Generate Cities List

In [5]:
# set up lists to hold reponse info
lat_lngs = []
cities = []


#create a set of random lat and long combos 
lats = np.random.uniform(lat_range[0], lat_range[1], size = 1400)
lngs = np.random.uniform(lng_range[0], lng_range[1], size = 1400)
lat_lngs = zip(lats,lngs)


# Loop through the list of cities and perform a request for data on each
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    if city not in cities: 
        cities.append(city)
        
len(cities)

561

In [6]:
#Build URL
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial" 
settings = {"units": "imperial", "appid": api_key}
url = f"{url}appid={api_key}&units={units}"

In [7]:
url

'http://api.openweathermap.org/data/2.5/weather?appid=aa93c53309c056c92eb6db60f5a5c869&units=imperial'

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).

In [8]:
# Create empty lists to append the API data into lists 
latitude = []
longitude = []
temp_max = []
humidity = []
cloud_coverage = []
wind_speed = []
city_name = []
country = []
datetime = []

# Start the call counter 
record_counter = 1

# Log file print statement
print(f"Start Data Retrieval Process")
print(f"-------------------------------")

#Loop through the cities in the city list 
for city in cities:  
    try: 
        response = requests.get(f"{url}&q={city}").json() 
        latitude.append(response["coord"]["lat"])
        longitude.append(response["coord"]["lon"])
        temp_max.append(response["main"]["temp_max"])
        humidity.append(response["main"]["humidity"])
        cloud_coverage.append(response["clouds"]["all"])
        wind_speed.append(response["wind"]["speed"])
        city_name.append(response["name"])
        country.append(response["sys"]["country"])
        datetime.append(response["dt"])
        city_record = response["name"]
        print(f"Processing Record {record_counter} | {city_record}")
        print(f"{url}&q={city}")
        
        record_counter= record_counter + 1        
        #set speed for API to download data
        time.sleep(0.50)
        
    # If city not found, skip 
    except:
        print("City not found")
    continue

Start Data Retrieval Process
-------------------------------
Processing Record 1 | Al Bardīyah
http://api.openweathermap.org/data/2.5/weather?appid=aa93c53309c056c92eb6db60f5a5c869&units=imperial&q=bardiyah
Processing Record 2 | Pemangkat
http://api.openweathermap.org/data/2.5/weather?appid=aa93c53309c056c92eb6db60f5a5c869&units=imperial&q=pemangkat
Processing Record 3 | Mar del Plata
http://api.openweathermap.org/data/2.5/weather?appid=aa93c53309c056c92eb6db60f5a5c869&units=imperial&q=mar del plata
Processing Record 4 | Bredasdorp
http://api.openweathermap.org/data/2.5/weather?appid=aa93c53309c056c92eb6db60f5a5c869&units=imperial&q=bredasdorp
Processing Record 5 | Mataura
http://api.openweathermap.org/data/2.5/weather?appid=aa93c53309c056c92eb6db60f5a5c869&units=imperial&q=mataura
Processing Record 6 | Jamestown
http://api.openweathermap.org/data/2.5/weather?appid=aa93c53309c056c92eb6db60f5a5c869&units=imperial&q=jamestown
Processing Record 7 | Port Alfred
http://api.openweathermap.or

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
# create a data frame from cities, lat, and temp
weather_dict = {
    "city": cities,
    "lat": lat,
    "temp": temp


latitude = []
longitude = []
temp_max = []
humidity = []
cloud_coverage = []
wind_speed = []
city_name = []
country = []
datetime = []

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression